In [1]:
import carla
import random
import re
import time

### Cliente

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

In [3]:
# world.__dir__()
for map_name in client.get_available_maps():
    print(map_name)
world.get_map().name

/Game/Carla/Maps/Town01
/Game/Carla/Maps/Town01_Opt
/Game/Carla/Maps/Town02
/Game/Carla/Maps/Town02_Opt
/Game/Carla/Maps/Town03
/Game/Carla/Maps/Town03_Opt
/Game/Carla/Maps/Town04
/Game/Carla/Maps/Town04_Opt
/Game/Carla/Maps/Town05
/Game/Carla/Maps/Town05_Opt
/Game/Carla/Maps/Town10HD
/Game/Carla/Maps/Town10HD_Opt


'Carla/Maps/Town10HD_Opt'

In [4]:
# client.load_world('Town04')
world = client.load_world("Town10HD_Opt")

In [5]:
def find_weather_presets():
    rgx = re.compile('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)')
    name = lambda x: ' '.join(m.group(0) for m in rgx.finditer(x))
    presets = [x for x in dir(carla.WeatherParameters) if re.match('[A-Z].+', x)]
    return [(getattr(carla.WeatherParameters, x), name(x)) for x in presets]

presets = find_weather_presets()
for idx, preset in enumerate(presets):
    print(idx, preset)
    if idx == 10:
        break

0 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF0ED0>, 'Clear Night')
1 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF0420>, 'Clear Noon')
2 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF0AE0>, 'Clear Sunset')
3 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF0F60>, 'Cloudy Night')
4 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF0780>, 'Cloudy Noon')
5 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF0B70>, 'Cloudy Sunset')
6 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF0660>, 'Default')
7 (<carla.libcarla.WeatherParameters object at 0x0000021BB3E3D300>, 'Dust Storm')
8 (<carla.libcarla.WeatherParameters object at 0x0000021BB3E3D270>, 'Hard Rain Night')
9 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF09C0>, 'Hard Rain Noon')
10 (<carla.libcarla.WeatherParameters object at 0x0000021BB3DF0DB0>, 'Hard Rain Sunset')


In [6]:
world.set_weather(presets[9][0])

In [7]:
# Pega a blueprint library and filtra só blueprints de carros
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
for idx, blueprint in enumerate(vehicle_blueprints):
    print(idx, blueprint.id)
    if idx == 10:
        break

0 vehicle.audi.a2
1 vehicle.nissan.micra
2 vehicle.audi.tt
3 vehicle.mercedes.coupe_2020
4 vehicle.bmw.grandtourer
5 vehicle.harley-davidson.low_rider
6 vehicle.ford.ambulance
7 vehicle.micro.microlino
8 vehicle.carlamotors.firetruck
9 vehicle.carlamotors.carlacola
10 vehicle.carlamotors.european_hgv


### Add vehicle to world

In [8]:
# Pega os spawn points do mapa
spawn_points = world.get_map().get_spawn_points()
# Tesla Cybertruck
vehicle = world.try_spawn_actor(list(vehicle_blueprints)[30], random.choice(spawn_points))
vehicle

In [9]:
vehicle_transform = vehicle.get_transform()
world.get_spectator().set_transform(vehicle_transform)

### Add sensors to car

In [10]:
SENSOR_TICK = 3

def get_cam_blueprint(world, vehicle_id):
    cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    cam_bp.set_attribute("image_size_x",str(400))
    cam_bp.set_attribute("image_size_y",str(300))
    cam_bp.set_attribute("fov",str(100))
    cam_bp.set_attribute("sensor_tick",str(SENSOR_TICK))
    return cam_bp


def get_lidar_blueprint(world, vehicle_id):
    lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast')
    lidar_bp.set_attribute('sensor_tick', str(SENSOR_TICK))
    lidar_bp.set_attribute('channels', '64')
    lidar_bp.set_attribute('points_per_second', '1120000')
    lidar_bp.set_attribute('upper_fov', '30')
    lidar_bp.set_attribute('range', '100')
    lidar_bp.set_attribute('rotation_frequency', '20')
    return lidar_bp

camera_init_trans = carla.Transform(carla.Location(z=2.3))
lidar_init_trans = carla.Transform(carla.Location(z=3.0))

In [11]:
# cria camera e 'anexa' ao carro
camera_bp = get_cam_blueprint(world, vehicle.id)
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
camera.listen(lambda image: image.save_to_disk(f"sensors/camera/vehicle-id-{vehicle.id}-frame-{image.frame}.png"))

# cria lidar e 'anexa' ao carro
lidar_bp = get_lidar_blueprint(world, vehicle.id)
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)
lidar.listen(lambda data: data.save_to_disk(f'sensors/lidar/vehicle-id-{vehicle.id}-frame-{data.frame}.ply'))

# adiciona ao Traffic manager
vehicle.set_autopilot(True)

In [ ]:
# Faz o spectator seguir o carro
spectator = world.get_spectator()
while True:
    transform = camera.get_transform()
    transform.location.z += 2.0
    spectator.set_transform(transform)
    time.sleep(0.0002)

In [17]:
# Remove o carro da simulação
vehicle.destroy()

### add a lot of vehicles

In [13]:
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

vehicle_num = 100

# add vehicle_num number of vehicles at random spawn_points
for i in range(vehicle_num):
    world.try_spawn_actor(list(vehicle_blueprints)[38], random.choice(spawn_points))
    #world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [14]:
# spawned_vehicles = world.get_actors().filter('*vehicle*')
spawned_vehicles = world.get_actors().filter('*cooper_s*')

In [15]:
for random_vehicle in spawned_vehicles:
    print(random_vehicle)
    # adiciona carros ao Traffic manager
    random_vehicle.set_autopilot(True)

Actor(id=50, type=vehicle.mini.cooper_s_2021)
Actor(id=51, type=vehicle.mini.cooper_s_2021)
Actor(id=52, type=vehicle.mini.cooper_s_2021)
Actor(id=53, type=vehicle.mini.cooper_s_2021)
Actor(id=54, type=vehicle.mini.cooper_s_2021)
Actor(id=55, type=vehicle.mini.cooper_s_2021)
Actor(id=56, type=vehicle.mini.cooper_s_2021)
Actor(id=57, type=vehicle.mini.cooper_s_2021)
Actor(id=58, type=vehicle.mini.cooper_s_2021)
Actor(id=59, type=vehicle.mini.cooper_s_2021)
Actor(id=60, type=vehicle.mini.cooper_s_2021)
Actor(id=61, type=vehicle.mini.cooper_s_2021)
Actor(id=62, type=vehicle.mini.cooper_s_2021)
Actor(id=63, type=vehicle.mini.cooper_s_2021)
Actor(id=64, type=vehicle.mini.cooper_s_2021)
Actor(id=65, type=vehicle.mini.cooper_s_2021)
Actor(id=66, type=vehicle.mini.cooper_s_2021)
Actor(id=67, type=vehicle.mini.cooper_s_2021)
Actor(id=68, type=vehicle.mini.cooper_s_2021)
Actor(id=69, type=vehicle.mini.cooper_s_2021)
Actor(id=70, type=vehicle.mini.cooper_s_2021)
Actor(id=71, type=vehicle.mini.coo

In [4]:
for random_vehicle in spawned_vehicles:
    random_vehicle.destroy()